In [1]:
import os, sys
PACKAGE_ROOT = os.path.dirname(os.path.abspath(""))
# print(PACKAGE_ROOT)
sys.path.insert(0, PACKAGE_ROOT)

import pandas as pd
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report

from commons import constants as C
from feature_engineering import generate_simple_word_features

# train data
train_df = pd.read_csv(r"C:\Users\sharm\Documents\nlp_assignment\ner_impls\data\train.csv")

# validation data
valid_df = pd.read_csv(r"C:\Users\sharm\Documents\nlp_assignment\ner_impls\data\valid.csv")

# test data
test_df = pd.read_csv(r"C:\Users\sharm\Documents\nlp_assignment\ner_impls\data\test.csv")

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# train data
agg_df = train_df.groupby(["sentence_id"]).agg( word_list=pd.NamedAgg(column="token", aggfunc=list), 
                                      tag_list=pd.NamedAgg(column="tag", aggfunc=list) ).reset_index()
X_train = list(agg_df.word_list.values)
y_train = list(agg_df.tag_list.values)

# validation data
agg_df = valid_df.groupby(["sentence_id"]).agg( word_list=pd.NamedAgg(column="token", aggfunc=list), 
                                      tag_list=pd.NamedAgg(column="tag", aggfunc=list) ).reset_index()
X_valid = list(agg_df.word_list.values)
y_valid = list(agg_df.tag_list.values)

# test data
agg_df = test_df.groupby(["sentence_id"]).agg( word_list=pd.NamedAgg(column="token", aggfunc=list), 
                                      tag_list=pd.NamedAgg(column="tag", aggfunc=list) ).reset_index()
X_test = list(agg_df.word_list.values)
y_test = list(agg_df.tag_list.values)

In [26]:

words = train_df["token"].values.tolist()
words = [generate_simple_word_features(word) for word in words]
tags = train_df["tag"].values.tolist()

In [22]:
# List down all the unique length of sentences and words. As expected, set of all the unqiue lengths of word is singleton.
# n_sentence = set()
# n_word = set()
# for sentence in X_train_wf:
#     n_sentence.add(len(sentence))
#     for word in sentence:
#         n_word.add(len(word))
#         # print(len(sentence), len(word))
# # print(n_sentence)
# print(n_word)

{6}


In [27]:
pred = cross_val_predict(RandomForestClassifier(n_estimators=20),
                         X=words, y=tags, cv=5)
report = classification_report(y_pred=pred, y_true=tags)
print(report)

c:\Users\sharm\anaconda3\envs\nlp_assignment\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\sharm\anaconda3\envs\nlp_assignment\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                         precision    recall  f1-score   support

B-AerospaceManufacturer       0.00      0.00      0.00       216
  B-AnatomicalStructure       0.00      0.00      0.00       388
              B-ArtWork       0.00      0.00      0.00       199
               B-Artist       0.09      0.00      0.00      3712
              B-Athlete       0.00      0.00      0.00      1793
      B-CarManufacturer       0.00      0.00      0.00       249
               B-Cleric       0.00      0.00      0.00       299
             B-Clothing       0.00      0.00      0.00       197
              B-Disease       0.00      0.00      0.00       372
                B-Drink       0.00      0.00      0.00       212
             B-Facility       0.02      0.00      0.00      1053
                 B-Food       0.00      0.00      0.00       362
      B-HumanSettlement       0.07      0.00      0.00      2617
     B-MedicalProcedure       0.00      0.00      0.00       242
   B-Medication/Vaccine 

c:\Users\sharm\anaconda3\envs\nlp_assignment\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## random forest with some more features context features

In [17]:
import os, sys
PACKAGE_ROOT = os.path.dirname(os.path.abspath(""))
# print(PACKAGE_ROOT)
sys.path.insert(0, PACKAGE_ROOT)

import pandas as pd
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline

from commons import constants as C
from feature_engineering import generate_simple_word_features
from feature_engineering import FeatureTransformer

# train data
train_df = pd.read_csv(r"C:\Users\sharm\Documents\nlp_assignment\ner_impls\data\train.csv")

# validation data
valid_df = pd.read_csv(r"C:\Users\sharm\Documents\nlp_assignment\ner_impls\data\valid.csv")

# test data
test_df = pd.read_csv(r"C:\Users\sharm\Documents\nlp_assignment\ner_impls\data\test.csv")

In [3]:
from models.memory_tagger import MemoryTagger
from sklearn.preprocessing import LabelEncoder
memory_tagger = MemoryTagger()
tag_encoder = LabelEncoder()
# all the words as a single list
words = train_df["token"].values.tolist()
# all the tags as single list 
tags = train_df["tag"].values.tolist()
# fit memory tagger
memory_tagger.fit([words], [tags])
# encode tags
tag_encoder.fit(tags)

LabelEncoder()

In [4]:
print(len(words), len(tags))

253011 253011


In [10]:
import warnings
warnings.filterwarnings("ignore")
ft = FeatureTransformer()
ft.fit(train_df, None)
train_df_transformed = ft.transform(train_df)

In [21]:
train_df_transformed[0]

array([ 0,  1,  0,  6,  0,  1,  3, 52, 66])

In [22]:
rfc = RandomForestClassifier(n_estimators=20, n_jobs=-1)
rfc.fit(train_df_transformed, tags)

RandomForestClassifier(n_estimators=20, n_jobs=-1)

In [40]:
X_valid = ft.transform(valid_df)
y_valid = valid_df["tag"].values.tolist()
y_pred = rfc.predict(X_valid)
print(y_pred)
print(len(y_pred))

['O' 'I-Athlete' 'O' ... 'O' 'O' 'O']
13323


In [41]:
# changing to MLB for calculating metrics report
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
yv = mlb.fit_transform([y_valid])
yp = mlb.transform([y_pred])
# yv
# mlb.classes_

In [47]:
y_pred

array(['O', 'I-Athlete', 'O', ..., 'O', 'O', 'O'], dtype='<U23')

In [48]:
from sklearn.metrics import classification_report
report = classification_report(y_true=y_valid, y_pred=y_pred, output_dict=True)#, target_names=mlb.classes_)
report_df = pd.DataFrame(report).transpose()
report_df

,precision,recall,f1-score,support
B-AerospaceManufacturer,0.818182,0.900000,0.857143,10.000000
B-AnatomicalStructure,0.750000,0.529412,0.620690,17.000000
B-ArtWork,0.500000,0.153846,0.235294,13.000000
B-Artist,0.462766,0.410377,0.435000,212.000000
B-Athlete,0.369231,0.303797,0.333333,79.000000
B-CarManufacturer,0.500000,0.461538,0.480000,13.000000
B-Cleric,0.444444,0.266667,0.333333,15.000000
B-Clothing,0.250000,0.100000,0.142857,10.000000
B-Disease,0.500000,0.444444,0.470588,18.000000
B-Drink,1.000000,0.454545,0.625000,11.000000
